# Section 2 - Quelques pas plus loin en R


## <span style="font-family:Calibri">2.3. Manipuler les données (avec dplyr)</span>

<font size=3 color=grey>
 2.3.1. Renommer des variables (rappel) <br>
 2.3.2. Selectionner des variables <b><font color=#3355BB size=3>SELECT </font></b><br>
 2.3.3. Créer de nouvelles variables <b><font color=#3355BB size=3>MUTATE </font></b><br>
 2.3.4. Filtrer <b><font color=#3355BB size=3>FILTER </font></b> <br>
 2.3.5. Trier <b><font color=#3355BB size=3>ARRANGE </font></b><br>
 2.3.6. Grouper <b><font color=#3355BB size=3>GROUP_BY </font></b> et <b><font color=#3355BB size=3>SUMMARISE </font></b> <br>
 2.3.7. Les jointures<br>
 2.3.8. l'opérateur "then" %>%<br>
</font>
<br><br><br>

<div style="border: 2px solid #1b70b8; padding: 3px; background-color: #c5ddf6; -moz-border-radius-topleft: 5px; -moz-border-radius-topright: 5px; -moz-border-radius-bottomright: 5px; -moz-border-radius-bottomleft: 5px;">
<b><font size = 3>Rappel des operateurs logiques et de selection </font></b><br>
 - ordre            : x >= 1 <br>
 - égalité          : x == 1 <br>
 - inégalité        : x != 1 <br>
 - appartenance     : x %in% ("a", "b") <br>
 - non appartenance : !(x %in% ("a", "b"))
 <br>
<b>et</b> est représenté par <b>&</b> <br>
<b>ou</b> est représenté par <b>|</b> <br>
<br>
</div>

<br>
# <span style="font-family:Calibri">2.3. Manipuler les données (avec dplyr) </span>

- package élaboré par Hadley Wickham 
- commpatible avec data.frame, data.table, MariaDB, SQLLite, MySQL...

https://github.com/hadley/dplyr
https://cran.rstudio.com/web/packages/dplyr/vignettes/introduction.html

<i>Nota : Dplyr utilise une technique nommée <u><b>Evaluation Non Standard (NSE)</u></b>.  
NSE est utile pour limiter la longueur de code, mais permet egalement de traduire en sous marin le code R en SQL.  

Néanomins, si NSE est très pratique pour l'usage interactif, il peut rendre dans certains la programmation plus complexe (si les "raccourcis" n'ont pas été pré-créés).  

Toutes les fonctions de dplyr, qui utilisent NSE, on egalement une version SE.  
Le nom de la fonction SE est identique à celui de NSE mais suffixé _.  
Par exemple, la version SE de summarise() est summarise_(), select()
<br>
vignette("nse") pour + d'infos </i>

In [ ]:
library(dplyr, verbose = FALSE, warn.conflicts = FALSE)

In [ ]:
set.seed(1)
N <- 50

DF <- data.frame(tmp.habitat = sample(c("appartement", "maison") , N, replace = T),
                 tmp.qualite = sample(LETTERS[(1:5)], N, replace = T),
                 tmp.capital = rpois(N, 100))

DF$tmp.ligne <- as.integer(rownames(DF) )   # nota : par defaut, le nom de la ligne est son numero. 
                                            #        si deja nommée, utiliser as.integer(rownames())

head(DF)
paste(nrow(DF), "lignes dans l'objet")

class(DF)

In [ ]:
set.seed(1)
(DF2 <- head(DF))



### 2.3.1. Renommer des variables (rappel)

- names(),
- colnames() 
- rownames() 

In [ ]:
names(DF)

In [ ]:
names(DF) <- c("habitat", "qualite", "capital", "ligne")
names(DF)

In [ ]:
names(DF2)
names(DF2) <- names(DF)
names(DF2)

<br>
### 2.3.2. Selectionner des variables <b><font color=#3355BB size=3>SELECT </font></b>
On peut facilement reproduire l'equivalent d'un <b>KEEP</b> ou d'un <b>DROP</b>, avec en outre la possibilité de test sur le nom de variables : <br>
<b>starts_with("")</b> permet de detecter les modalités suivant leur prefixe (equivalent de VAR: en SAS)<br>
<b>ends_with("")</b>   permet de detecter les modalités suivant leur suffixe<br>
<b>contains("")</b>    permet de detecter les modalités contenant le pattern indiqué


In [ ]:
select(DF2, c(ligne, habitat))

In [ ]:
names(DF2)

In [ ]:
select(DF2, -c(ligne, habitat))

In [ ]:
select(DF2, contains("a"))  ## Cette terminologie est tres pratique en relation avec les conventions de nommage Blueprint

Exemple de select_ en <b>Evaluation Standard (SE)</b> :  
Cette méthode est utile si l'on recupere les noms des variables par code par exemple,
et qu'elles sont de fait encadrés par ""

In [ ]:
select(DF2, keep)

In [ ]:
keep <- c("ligne", "habitat")
select_(DF2, .dots = keep)

In [ ]:
drop <- paste0("-", c("ligne", "habitat"))
select_(DF2, .dots = drop)

### 2.3.3. Créer de nouvelles variables <b><font color=#3355BB size=3>MUTATE </font></b>
Construction de nouvelles variables a la volée. Juxtaposition possible pour creer plusieurs variables en 1 coup.<br>
Pour mettre a jour le dataframe, il faut bien rediriger le resultat du mutate vers DT2 !

In [ ]:
(DF2 <- mutate(DF2, capital_moins_ligne = capital-ligne))

In [ ]:
 a <- names(DF2)[5]

In [ ]:
b <- names(DF2)[(1:4)]

In [ ]:
DF2[, c(a, b)]

In [ ]:
mutate(DF2, qualite2 = paste0(qualite, ligne), 
            rand = runif(nrow(DF2)),
            capital_cr = (capital - mean(capital))/sd(capital)
      )

### 2.3.4. Filtrer <b><font color=#3355BB size=3>FILTER </font></b> 

In [ ]:
filter(DF, qualite %in% c("B", "D") & capital >=100)


### 2.3.5. Trier <b><font color=#3355BB size=3>ARRANGE </font></b>

In [ ]:
arrange(DF2, qualite)

In [ ]:
arrange(DF2, qualite, desc(capital - ligne))

### 2.3.6. Grouper <b><font color=#3355BB size=3>GROUP_BY </font></b> et <b><font color=#3355BB size=3>SUMMARISE </font></b> 

 Le principe est de construire le vecteur de variables d'agregation grace a <b>group_by</b> puis l'utiliser dans <b>summarise</b><br>
 <br><u>Rappel de fonctions de calculs</u> : <br>
 * min(x), median(x), max(x), quantile(x, p)
 * n(), n_distinct(), sum(x), mean(x)
 * sum(x > 10), mean(x > 10)
 * sd(x), var(x), iqr(x), mad(x)

In [ ]:
summarise(DF, total = sum(capital))

In [ ]:
by_habitat <- group_by(DF, habitat)

In [ ]:
summarise(by_habitat, total = sum(capital))

In [ ]:
summarise(group_by(DF, habitat), total = sum(capital))

In [ ]:
by_habitat_qualite <- group_by(DF, habitat, qualite)

summarise(by_habitat_qualite,
    capital.moy     = round(mean(capital), 1),
    capital.med     = median(capital),
    capital.q5      = quantile(capital, 0.05),
    capital.over100 = sum(capital > 100)
)

### 2.3.7. Les jointures

<table border = 1 align = left>
<tr>
 <td>inner_join(x, y, by = c("clef1", "clef2"))
 </td>
 <td>n'inclut que les lignes présentes dans x et y
 </td>
</tr>
<tr>
 <td>left_join(x, y, ..)
 </td>
 <td>inclut toutes les lignes de x, et matchent celles de y quand dispo, sinon NA
 </td>
</tr>
<tr>
 <td>semi_join(x, y, ..)
 </td>
 <td>inclut les lignes de x qui sont egalement dans y
 </td>
</tr>
<tr>
 <td>anti_join(x, y, ..)
 </td>
 <td>inclut les lignes de x qui ne sont pas dans y
 </td>
</tr>
</table>


Nota : si les variables ont des noms differents, la syntaxe est la suivante :  

<div style="border: 0px solid #1b70b8; padding: 3px; background-color: #1b70b8;">
<font color = "white" size = 3> inner_join</font>

<img src="http://upload.dinhosting.fr/V/D/W/join_inner_join.jpg" height="30" align="left">

<div style="border: 0px solid #1b70b8; padding: 3px; background-color: #1b70b8;">
<font color = "white" size = 3> left_join</font>


<img src="http://upload.dinhosting.fr/Q/V/O/join_left_join.jpg" height="50" align="left">

<div style="border: 0px solid #1b70b8; padding: 3px; background-color: #1b70b8;">
<font color = "white" size = 3> anti_join</font>

<img src="http://upload.dinhosting.fr/k/v/w/join_anti_join.jpg" height="50" align="left">

<div style="border: 0px solid #1b70b8; padding: 3px; background-color: #1b70b8;">
<font color = "white" size = 3> semi_join</font>


<img src="http://upload.dinhosting.fr/i/z/o/join_semi_join.jpg" height="50" align="left">

<br><br><br>
### 2.3.7. l'opérateur "then" %>%

- issu de library(magrittr)
- idem <b>|</b> sur unix.
- enchaine au lieu d'imbriquer les opérations N-1 => N
<br>
<br>
Imaginons que l'on souhaite 
 - prendre les observations dont le capital est >100
 - calculer la moyenne et l'ecart type du capital par habitat x qualité, ainsi que le nombre d'obs agregées
 - et ne conserver que les lignes telles que ce nombre d'obs agregé est >=5

In [ ]:
   filter(
       summarise(
            group_by(filter(DF, capital > 100), habitat, qualite), 
            capital.moy = mean(capital),
            capital.sd  = sd(capital),
            nobs        = n()
                ),
          nobs>5)



In [ ]:
DF                                    %>%
filter(capital > 100)                 %>%
group_by(habitat, qualite)            %>%
summarise(capital.moy = mean(capital),
          capital.sd  = sd(capital),
          nobs = n())                 %>%
filter(nobs >5)


<br><br><br>
<br>
<font color = "00FF44" size=5><b>A VOUS DE JOUER !!</b></font>



1/ Charger le fichier des rabais Auto et verifier son contenu  <br><br>
2/ Construire une variable primeht_tech = 100*primeht/rabais - 
   N'afficher que les 5 premieres lignes et les 3 colonnes considerées primeht, rabais et primeht_tech <br><br>
3/ Calculer le rabais moyen, les primes ht et ht_tech moyenne et les effectifs par age du conducteur, trié par rabais croissant <br><br>
4/ charger le fichier SRA201512.csv et effectuer la jointure par COD_auto.<br>
   Nommez la table resultants DF.rabais.SRA <br><br>
5/ Effectuez la même opération qu'au point 3/ restreint aux vehicules de classe de prix >=R en région Sud-Est (67)<br>
6/ Sauver la table DF.rabais.SRA au format .Rdata

In [ ]:
## 1/ Charger le fichier .Rdata
load("C:/R/05_Training/Axa_Training/Data/rabaisAUTO_corr.Rdata", verbose = TRUE)
library(dplyr, warn.conflict = FALSE)

In [ ]:
## Rappel du contenu du dataset
str(DF.rabais)
class(DF.rabais)

In [ ]:
## 2/ Construire une variable prime technique = 100*primeht/rabais 
DF.rabais <- DF.rabais %>%
mutate(primeht_tech = 100*primeht/rabais) 


In [ ]:
# affichage
head(select(DF.rabais, primeht, rabais, primeht_tech), 5)

In [ ]:
## 3/ Calculer le rabais moyen, les primes ht et ht_tech moyenne et les effectifs par age du conducteur , 
##    trié par rabais croissant

DF.rabais %>%
group_by(agecond2) %>%
summarise(moy_rabais = mean(rabais), 
          moy_prime = mean(primeht), 
          moy_prime_tech = mean(primeht_tech), 
          effectif = n())  %>%
arrange(moy_rabais)

In [ ]:
## 4/ charger le fichier SRA201512.csv et effectuer la jointure par COD_auto 
library(data.table, warn.conflict = FALSE)
SRA <- fread("C:/R/05_Training/Axa_Training/Data/SRA201512.csv")
str(SRA)

In [ ]:
str(DF.rabais)

In [ ]:
DF.rabais.SRA <- left_join(DF.rabais, SRA, by = c("GTA" = "COD_auto"))
dim(DF.rabais)
dim(DF.rabais.SRA)

In [ ]:
LETTERS[(1:18)]
letters[(1:4)]

In [ ]:
## 5/ Effectuez la même opération qu'au point 3/ 
## restreint aux vehicules de classe de prix >=R en région Sud-Est (67)

DF.rabais.SRA %>%
filter(nreg == "67", ! (CLASS_PRIX %in% LETTERS[(1:18)] ))  %>%
group_by(agecond2) %>%
summarise(moy_rabais = mean(rabais), 
          moy_prime = mean(primeht), 
          moy_prime_tech = mean(primeht_tech), 
          effectif = n())  %>%
arrange(moy_rabais)

In [ ]:
save(DF.rabais.SRA, file = "C:/R/05_Training/Axa_Training/Data/DF.rabais.SRA.Rdata")